In [ ]:
# pip install azureml-core
# pip install azureml azureml-sdk 
# pip install azureml-mlflow

In [1]:
from azureml.core import Workspace

c:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
# create a new workspace or access existing workspace
try:
    ws = Workspace.create(name="aml-anshu2",
                            subscription_id="84a5808b-5549-459a-98f2-f102e84fa1bb",
                            resource_group="EYMAY",
                            create_resource_group=True,
                            location='westus')

except:
    # access existing workspace
    ws = Workspace.get(name="aml-anshu2",
                        subscription_id="84a5808b-5549-459a-98f2-f102e84fa1bb",
                        resource_group="EYMAY")
finally:
    print(ws)

Deploying KeyVault with name amlanshukeyvault4c67362f.
Deploying StorageAccount with name amlanshustorage4145e8942.
Deployed KeyVault with name amlanshukeyvault4c67362f. Took 28.26 seconds.
Deployed StorageAccount with name amlanshustorage4145e8942. Took 30.73 seconds.
Deploying Workspace with name aml-anshu2.
Deploying AppInsights with name amlanshuinsightsc6593522.
Deployed AppInsights with name amlanshuinsightsc6593522. Took 33.23 seconds.
Deployed Workspace with name aml-anshu2. Took 24.73 seconds.
Workspace.create(name='aml-anshu2', subscription_id='84a5808b-5549-459a-98f2-f102e84fa1bb', resource_group='EYMAY')


In [5]:
ws

Workspace.create(name='aml-anshu2', subscription_id='84a5808b-5549-459a-98f2-f102e84fa1bb', resource_group='EYMAY')

In [6]:
import pandas  as pd
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 
import mlflow 
import mlflow.sklearn
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [7]:
# data processing
df = pd.read_csv(r"Bank_churn_modelling.csv")
x = df[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'IsActiveMember']]
y =df['Exited']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,stratify=y,random_state=5)

transformer = ColumnTransformer([('ohe',OneHotEncoder(drop="first"),[1,2]),],remainder='passthrough')
#transformer.fit(xtrain)

In [8]:
ws.get_mlflow_tracking_uri()

'azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-anshu2?'

In [9]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [10]:
mlflow.set_experiment("bank_churn")

2023/05/11 09:09:22 INFO mlflow.tracking.fluent: Experiment with name 'bank_churn' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1683776363520, experiment_id='e15fa797-2b1c-448e-a5bf-44d66e986104', last_update_time=None, lifecycle_stage='active', name='bank_churn', tags={}>

In [11]:
evaldata = xtest
evaldata['label'] = ytest

In [21]:

mlflow.sklearn.autolog()
with mlflow.start_run():
    model_pipeline = Pipeline([("transformer",transformer),
                                ("model",DecisionTreeClassifier(criterion='gini',min_samples_leaf=20,max_depth=8,
                               class_weight='balanced',random_state=5))])
    model_pipeline.fit(xtrain,ytrain)

    model_uri = mlflow.get_artifact_uri("model")
    #model evaluation

    result = mlflow.evaluate(model=model_uri,data=evaldata,
    targets='label',model_type='classifier',
    evaluators=['default'],
    evaluator_config={"default":{"metric_prefix":"test_","log_model_explainability":False}}
    )

2023/05/11 10:23:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\anshu\AppData\Roaming\Python\Python310\site-packages\mlflow\models\signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/05/11 10:24:14 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/05/11 10:24:14 INFO mlfl

In [12]:
from mlflow.models.signature import infer_signature,ModelSignature
from mlflow.types.schema import Schema, ColSpec

In [13]:

mlflow.sklearn.autolog()
with mlflow.start_run():
    model_pipeline = Pipeline([("transformer",transformer),
                                ("model",DecisionTreeClassifier(criterion='gini',min_samples_leaf=20,max_depth=8,
                               class_weight='balanced',random_state=5))])
    model_pipeline.fit(xtrain,ytrain)

    # specify signature manually
    input_schema = Schema([
        ColSpec("double",'CreditScore'),
        ColSpec("string",'Geography'),
        ColSpec("string",'Gender'),
        ColSpec("double",'Age'),
        ColSpec("double",'Balance'),
        ColSpec("double",'NumOfProducts'),
        ColSpec("integer",'IsActiveMember'),
    ])
    output_schema = Schema([ColSpec("integer",'Exited'),])
    signature2 = ModelSignature(inputs=input_schema,outputs=output_schema)

    example = {"CreditScore":652.0,"Geography":'France',"Gender":"Male","Age":45.0,
                "Balance":152432.0,"NumOfProducts":2.0,"IsActiveMember":0}
    mlflow.sklearn.log_model(model_pipeline,"churn_model",
                                signature = signature2,
                                input_example=example
                                )

    model_uri = mlflow.get_artifact_uri("churn_model")
    #model evaluation

    result = mlflow.evaluate(model=model_uri,data=evaldata,
    targets='label',model_type='classifier',
    evaluators=['default'],
    evaluator_config={"default":{"metric_prefix":"test_"}},
    )

c:\ProgramData\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/05/11 09:09:48 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/05/11 09:09:49 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.
2023/05/11 09:10:04 WARNING mlflow.models.evaluation.default_evaluator: Skip logging model explainability insights because the shap explainer None requires all feature values to be numeric, and each feature column must only contain scalar values.


<Figure size 1050x700 with 0 Axes>

In [14]:
last_run = mlflow.last_active_run()
runid = last_run.info.run_id
runid

'76d3fa9b-71e1-440e-bc84-daca32c2ca4a'

In [15]:
# model registration
model_uri = f"runs:/{runid}/churn_model"
model_reg_name = "churn_prediction"
mlflow.register_model(model_uri,model_reg_name)

Successfully registered model 'churn_prediction'.
2023/05/11 09:10:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: churn_prediction, version 1
Created version '1' of model 'churn_prediction'.


<ModelVersion: aliases=[], creation_timestamp=1683776408445, current_stage='None', description='', last_updated_timestamp=1683776408445, name='churn_prediction', run_id='76d3fa9b-71e1-440e-bc84-daca32c2ca4a', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-anshu2/experiments/e15fa797-2b1c-448e-a5bf-44d66e986104/runs/76d3fa9b-71e1-440e-bc84-daca32c2ca4a/artifacts/churn_model', status='READY', status_message='', tags={}, user_id='', version='1'>

## Deployment over ACI

In [16]:
import json
from mlflow.deployments import get_deploy_client
deploy_config_path = "aciconfig.json"

client = get_deploy_client(ws.get_mlflow_tracking_uri())


In [19]:
# deployment configuration
config = {"deploy-config-file":deploy_config_path}
model_name = "churn_prediction"
model_version=1

In [20]:
webservice = client.create_deployment(model_uri = f"models:/{model_name}/{model_version}",
                            config=config,
                            name="churn-aci")

Running
2023-05-11 09:12:11+05:30 Creating Container Registry if not exists....................
2023-05-11 09:22:11+05:30 Registering the environment.
2023-05-11 09:22:14+05:30 Building image......................
2023-05-11 09:32:39+05:30 Generating deployment configuration.
2023-05-11 09:32:40+05:30 Submitting deployment to compute.
2023-05-11 09:32:45+05:30 Checking the status of deployment churn-aci...
2023-05-11 09:33:57+05:30 Checking the status of inference endpoint churn-aci..
Succeeded
AciService creation operation finished, operation "Succeeded"


In [20]:
webservice

{'name': 'churn-aci',
 'description': None,
 'tags': {},
 'properties': {'hasInferenceSchema': 'False',
  'hasHttps': 'False',
  'authEnabled': 'False'},
 'state': 'Healthy',
 'createdTime': '2023-05-10T07:08:45.608845+00:00',
 'updatedTime': '2023-05-10T07:08:45.608845+00:00',
 'error': None,
 'computeType': 'ACI',
 'workspaceName': 'aml-anshu',
 'imageId': None,
 'imageDigest': None,
 'imageDetails': None,
 'scoringUri': 'http://d676cf39-7576-45d1-81b6-9f73bca86469.westus.azurecontainer.io/score',
 'createdBy': {'userObjectId': '7501f233-1f33-4666-bc5d-f8e6add99ab0',
  'userPuId': '1003BFFD8FA46F19',
  'userIdp': 'live.com',
  'userAltSecId': '1:live.com:0003BFFDCDDF08B3',
  'userIss': 'https://sts.windows.net/13a86542-2185-4187-8e07-7512f5525c55/',
  'userTenantId': '13a86542-2185-4187-8e07-7512f5525c55',
  'userName': 'Anshu Pandey'},
 'containerResourceRequirements': {'cpu': 0.1, 'memoryInGB': 0.5},
 'location': 'westus',
 'authEnabled': False,
 'sslEnabled': False,
 'appInsightsE

In [21]:
webservice['scoringUri']

'http://d676cf39-7576-45d1-81b6-9f73bca86469.westus.azurecontainer.io/score'

In [30]:
webservice['modelDetails'][0]['properties']['model_json']

'{"run_id": "1352a106-f9f0-42d5-a13e-2749d5339050", "artifact_path": "churn_model", "utc_time_created": "2023-05-10 06:48:47.945834", "flavors": {"python_function": {"model_path": "model.pkl", "predict_fn": "predict", "loader_module": "mlflow.sklearn", "python_version": "3.10.9", "env": {"conda": "conda.yaml", "virtualenv": "python_env.yaml"}}, "sklearn": {"pickled_model": "model.pkl", "sklearn_version": "1.2.1", "serialization_format": "cloudpickle", "code": null}}, "model_uuid": "3593329c5df74a9c9322cc72d9e8fff7", "mlflow_version": "2.3.1", "signature": {"inputs": "[{\\"name\\": \\"CreditScore\\", \\"type\\": \\"double\\"}, {\\"name\\": \\"Geography\\", \\"type\\": \\"string\\"}, {\\"name\\": \\"Gender\\", \\"type\\": \\"string\\"}, {\\"name\\": \\"Age\\", \\"type\\": \\"double\\"}, {\\"name\\": \\"Balance\\", \\"type\\": \\"double\\"}, {\\"name\\": \\"NumOfProducts\\", \\"type\\": \\"double\\"}, {\\"name\\": \\"IsActiveMember\\", \\"type\\": \\"integer\\"}]", "outputs": "[{\\"name\\